In [1]:
import re 

from langchain_openai import OpenAIEmbeddings
from openai import OpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders  import PyPDFLoader
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA



from langchain_community.vectorstores import FAISS

### Langgraph docs loader


In [2]:
from langchain_community.document_loaders  import PyPDFLoader


loader = PyPDFLoader("./data/PERMENPAN NOMOR 38 TAHUN 2017.pdf")
documents = loader.load()

In [3]:
documents[0]

Document(metadata={'producer': 'PyPDF', 'creator': 'PyPDF', 'creationdate': '', 'source': './data/PERMENPAN NOMOR 38 TAHUN 2017.pdf', 'total_pages': 108, 'page': 0, 'page_label': '1'}, page_content='PERATURAN MENTERI PENDAYAGUNAAN APARATUR NEGARA DAN\nREFORMASI BIROKRASI REPUBLIK INDONESIA\nNOMOR 38 TAHUN 2017\nTENTANG\nSTANDAR KOMPETENSI JABATAN APARATUR SIPIL NEGARA\nDENGAN RAHMAT TUHAN YANG MAHA ESA\nMENTERI PENDAYAGUNAAN APARATUR NEGARA\nDAN REFORMASI BIROKRASI REPUBLIK INDONESIA,\nMenimbang : bahwa untuk melaksanakan ketentuan Pasal 55 ayat (5),\nPasal 109 ayat (4) dan ayat (5) dan Pasal 166 ayat (2)\nPeraturan Pemerintah Nomor 11 Tahun 2017 tentang\nManajemen Pegawai Negeri Sipil, perlu menetapkan Peraturan\nMenteri Pendayagunaan Aparatur Negara dan Reformasi\nBirokrasi tentang Standar Kompetensi Jabatan Aparatur Sipil\nNegara;\nMengingat : 1. Undang-Undang Nomor 5 Tahun 2014 tentang Aparatur\nSipil Negara (Lembaran Negara Republik Indonesia Tahun\n2014 Nomor 6, Tambahan Lembaran

In [4]:
import pprint

pprint.pp(documents[0].metadata)

{'producer': 'PyPDF',
 'creator': 'PyPDF',
 'creationdate': '',
 'source': './data/PERMENPAN NOMOR 38 TAHUN 2017.pdf',
 'total_pages': 108,
 'page': 0,
 'page_label': '1'}


### Preprocessing

In [6]:
import re 
def preprocess(text): 
    # Menghilangkan footer nomor halaman seperti "- 5 -"
    text = re.sub(r'-\s*\d+\s*-', '', text)
    text = re.sub(r'\n\s*\n', '\n\n', text)
    text = re.sub(r'\n(?!\n)', ' ', text)
    text = re.sub(r' +', ' ', text)

    return text.strip().lower()

for doc in documents:
    doc.page_content = preprocess(doc.page_content) 



In [7]:
documents[0:5]

[Document(metadata={'producer': 'PyPDF', 'creator': 'PyPDF', 'creationdate': '', 'source': './data/PERMENPAN NOMOR 38 TAHUN 2017.pdf', 'total_pages': 108, 'page': 0, 'page_label': '1'}, page_content='peraturan menteri pendayagunaan aparatur negara dan reformasi birokrasi republik indonesia nomor 38 tahun 2017 tentang standar kompetensi jabatan aparatur sipil negara dengan rahmat tuhan yang maha esa menteri pendayagunaan aparatur negara dan reformasi birokrasi republik indonesia, menimbang : bahwa untuk melaksanakan ketentuan pasal 55 ayat (5), pasal 109 ayat (4) dan ayat (5) dan pasal 166 ayat (2) peraturan pemerintah nomor 11 tahun 2017 tentang manajemen pegawai negeri sipil, perlu menetapkan peraturan menteri pendayagunaan aparatur negara dan reformasi birokrasi tentang standar kompetensi jabatan aparatur sipil negara; mengingat : 1. undang-undang nomor 5 tahun 2014 tentang aparatur sipil negara (lembaran negara republik indonesia tahun 2014 nomor 6, tambahan lembaran negara republik

In [8]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,    
    chunk_overlap=200,  
    length_function=len
)



docs_split = text_splitter.split_documents(documents)

In [9]:
docs_split[:10]

[Document(metadata={'producer': 'PyPDF', 'creator': 'PyPDF', 'creationdate': '', 'source': './data/PERMENPAN NOMOR 38 TAHUN 2017.pdf', 'total_pages': 108, 'page': 0, 'page_label': '1'}, page_content='peraturan menteri pendayagunaan aparatur negara dan reformasi birokrasi republik indonesia nomor 38 tahun 2017 tentang standar kompetensi jabatan aparatur sipil negara dengan rahmat tuhan yang maha esa menteri pendayagunaan aparatur negara dan reformasi birokrasi republik indonesia, menimbang : bahwa untuk melaksanakan ketentuan pasal 55 ayat (5), pasal 109 ayat (4) dan ayat (5) dan pasal 166 ayat (2) peraturan pemerintah nomor 11 tahun 2017 tentang manajemen pegawai negeri sipil, perlu menetapkan peraturan menteri pendayagunaan aparatur negara dan reformasi birokrasi tentang standar kompetensi jabatan aparatur sipil negara; mengingat : 1. undang-undang nomor 5 tahun 2014 tentang aparatur sipil negara (lembaran negara republik indonesia tahun 2014 nomor 6, tambahan lembaran negara republik

### Configuration

In [13]:
# LLM Configuration - Using OpenRouter (reliable cloud LLM service)
API_KEY = "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"  # OpenRouter API key
BASE_URL = "https://openrouter.ai/api/v1"
LLM_MODEL = "mistralai/mistral-7b-instruct-v0.2"  # demo
EMBEDDING_MODEL = "qwen/qwen3-embedding-8b"  # Model embedding Qwen 3
MAX_TOKENS = 256

In [14]:
client = OpenAI(base_url=BASE_URL,
                api_key=API_KEY, 
                timeout=30.0,
                max_retries=3)

### Embedding model

In [15]:

from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

embedding_model = OpenAIEmbeddings(
    base_url=BASE_URL,
    api_key=API_KEY,
    model=EMBEDDING_MODEL 
)


db = FAISS.from_documents(docs_split, embedding_model)

### Instruct model

In [16]:
from langchain_openai import ChatOpenAI
llm_chat = ChatOpenAI(
    base_url=BASE_URL,
    api_key=API_KEY, 
    model=LLM_MODEL, 
    temperature=0.7,
    max_tokens=256,
    streaming=True,
    verbose=True,
)

### Retrieval - Generation

In [17]:
retriever = db.as_retriever(
    search_type = "similarity",
    search_kwargs = {"k": 10 }
)

# option = mmr or similarity_with_threshold
# retriever_mmr = db.as_retriever(
#     # Ganti search_type menjadi "mmr"
#     search_type="mmr",
#     search_kwargs={
#         "k": 10,           
#         "fetch_k": 20,   
#         "lambda_mult": 0.5 
#     }
# )

query = """Apa inti dalam PERATURAN MENTERI PENDAYAGUNAAN APARATUR NEGARA
 DAN REFORMASI BIROKRASI TENTANG STANDAR
 KOMPETENSI JABATAN APARATUR SIPIL NEGARA ?"""

retrieved_docs = retriever.get_relevant_documents(query)

print(f"Dokumen terambil dari k :{retrieved_docs}")

C:\Users\erwin\AppData\Local\Temp\ipykernel_17616\1728485176.py:21: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(query)


Dokumen terambil dari k :[Document(id='cfb2b86e-6249-4100-a81a-12bc785fb325', metadata={'producer': 'PyPDF', 'creator': 'PyPDF', 'creationdate': '', 'source': './data/PERMENPAN NOMOR 38 TAHUN 2017.pdf', 'total_pages': 108, 'page': 12, 'page_label': '13'}, page_content='standar kompetensi jabatan yang ditetapkan oleh menteri menjadi standar dalam menyelenggarakan manajemen aparatur sipil negara yang berlaku secara nasional. c. ruang lingkup ruang lingkup penyusunan standar kompetensi yang diatur dalam peraturan menteri ini meliputi: 1. pedoman pembentukan dan tugas tim penyusun standar kompetensi; 2. pedoman dan tata cara penyusunan standar kompetensi jabatan dan persyaratan jabatan; 3. pedoman dan tata cara penetapan standar kompetensi jabatan. 4. pedoman pembentukan dan tugas tim penyusun kamus kompetensi teknis; dan 5. pedoman dan tata cara penyusunan kamus kompetensi teknis; d. pengertian dalam peraturan menteri ini yang dimaksud dengan: 1. standar kompetensi jabatan aparatur sipil 

In [18]:
#Prompt
from langchain.prompts import PromptTemplate

# Template ini akan diisi dengan dokumen dari retriever dan pertanyaan dari query
prompt_template = """
Gunakan potongan-potongan konteks berikut untuk menjawab pertanyaan pengguna.
Jawablah secara ringkas dan jelas dalam Bahasa Indonesia.
Jika Anda tidak tahu jawabannya berdasarkan konteks yang diberikan, katakan saja bahwa Anda tidak dapat menemukan informasinya, jangan mencoba mengarang jawaban.

KONTEKS:
{context}

PERTANYAAN:
{question}

JAWABAN YANG MEMBANTU:
"""

CUSTOM_PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [19]:
from langchain.chains import RetrievalQA
rag_chain = RetrievalQA.from_chain_type(
    llm = llm_chat,
    chain_type = "stuff", 
    retriever = retriever,
    chain_type_kwargs = {"prompt": CUSTOM_PROMPT},
    return_source_documents=True
)

In [20]:
# Menjalankan query melalui RAG Chain
response = rag_chain.invoke(query)

print("\nPERTANYAAN:")
print(query)

print("\nJAWABAN YANG DIHASILKAN LLM:")
print(response['result'])

print("\nDOCUMENT:")
print(response['source_documents'])


retrieved_docs


PERTANYAAN:
Apa inti dalam PERATURAN MENTERI PENDAYAGUNAAN APARATUR NEGARA
 DAN REFORMASI BIROKRASI TENTANG STANDAR
 KOMPETENSI JABATAN APARATUR SIPIL NEGARA ?

JAWABAN YANG DIHASILKAN LLM:
 Peraturan Menteri Pendayagunaan Aparatur Negara dan Reformasi Birokrasi mencakup inti tentang standar kompetensi jabatan aparatur sipil negara. Standar kompetensi ini mendefinisikan deskripsi pengetahuan, keterampilan, dan perilaku yang diperlukan seorang aparatur sipil negara dalam melaksanakan tugas jabatan. Standar ini ditetapkan oleh Menteri dan berlaku secara nasional.

DOCUMENT:
[Document(id='cfb2b86e-6249-4100-a81a-12bc785fb325', metadata={'producer': 'PyPDF', 'creator': 'PyPDF', 'creationdate': '', 'source': './data/PERMENPAN NOMOR 38 TAHUN 2017.pdf', 'total_pages': 108, 'page': 12, 'page_label': '13'}, page_content='standar kompetensi jabatan yang ditetapkan oleh menteri menjadi standar dalam menyelenggarakan manajemen aparatur sipil negara yang berlaku secara nasional. c. ruang lingkup r

[Document(id='cfb2b86e-6249-4100-a81a-12bc785fb325', metadata={'producer': 'PyPDF', 'creator': 'PyPDF', 'creationdate': '', 'source': './data/PERMENPAN NOMOR 38 TAHUN 2017.pdf', 'total_pages': 108, 'page': 12, 'page_label': '13'}, page_content='standar kompetensi jabatan yang ditetapkan oleh menteri menjadi standar dalam menyelenggarakan manajemen aparatur sipil negara yang berlaku secara nasional. c. ruang lingkup ruang lingkup penyusunan standar kompetensi yang diatur dalam peraturan menteri ini meliputi: 1. pedoman pembentukan dan tugas tim penyusun standar kompetensi; 2. pedoman dan tata cara penyusunan standar kompetensi jabatan dan persyaratan jabatan; 3. pedoman dan tata cara penetapan standar kompetensi jabatan. 4. pedoman pembentukan dan tugas tim penyusun kamus kompetensi teknis; dan 5. pedoman dan tata cara penyusunan kamus kompetensi teknis; d. pengertian dalam peraturan menteri ini yang dimaksud dengan: 1. standar kompetensi jabatan aparatur sipil negara yang selanjutnya d

### Rag untuk penilaian


In [22]:
llm_assessor = ChatOpenAI(
    base_url=BASE_URL,
    api_key=API_KEY,
    model=LLM_MODEL,
    temperature=0.3, 
    max_tokens=512,
    streaming=False,
    verbose=False,
)

In [23]:
from prompt.prompt import MANAGERIAL_ASSESSMENT_PROMPT
from langchain.chains import LLMChain


# LLM CHAIN 
assesment_chain =  LLMChain(
    llm = llm_assessor, 
    prompt= MANAGERIAL_ASSESSMENT_PROMPT
)


def competency_asessment(nama, jabatan, jawaban_test, kompetensi, level_target) : 
    query = f"kompetensi {kompetensi} level {level_target} indikator perilaku"
    relevant_docs = retriever.get_relevant_documents(query)
    context = "\n".join([doc.page_content for doc in relevant_docs])

    assesment_chain_result = assesment_chain.invoke({
        "context": context,
        "nama": nama,
        "jabatan": jabatan,
        "jawaban": jawaban_test,
        "kompetensi": kompetensi,
        "level_target": level_target
    })

    return {
        "hasil": assesment_chain_result['text'],
        "sumber": relevant_docs  
    }
    


C:\Users\erwin\AppData\Local\Temp\ipykernel_17616\1087566828.py:6: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  assesment_chain =  LLMChain(


In [ ]:
test_cases = [
    {
        "nama": "Budi Santoso",
        "jabatan": "Kepala Bagian Pengadaan Barang dan Jasa",
        "jawaban_test": """
        Dalam menyelesaikan konflik tim, saya pertama-tama mengumpulkan semua pihak 
        untuk mendengarkan perspektif masing-masing. Saya menggunakan pendekatan win-win solution 
        dengan mencari titik temu yang menguntungkan semua pihak. Setelah mencapai kesepakatan, 
        saya memastikan implementasinya dengan monitoring berkala dan evaluasi hasil.
        """,
        "kompetensi": "Kerjasama",
        "level_target": 3
    },
    {
        "nama": "Siti Rahayu",
        "jabatan": "Kepala Biro Umum",
        "kompetensi": "Pengambilan Keputusan", 
        "jawaban_test": """
        Untuk menyusun perencanaan SDM, saya melakukan analisis kebutuhan berdasarkan 
        rencana strategis organisasi. Saya menggunakan data historis dan proyeksi kebutuhan 
        untuk menentukan jumlah dan kualifikasi pegawai yang dibutuhkan. 
        Saya juga mempertimbangkan berbagai alternatif solusi dan melakukan analisis risiko 
        sebelum mengambil keputusan final yang menguntungkan organisasi.
        """,
        "level_target": 4
    },
    {
        "nama": "Ani Suryani",
        "jabatan": "Inspektur Utama",
        "jawaban_test": """
        Ketika menemukan adanya potensi penyalahgunaan wewenang di laporan, saya langsung
        melakukan verifikasi silang terhadap data dan peraturan yang berlaku tanpa memandang
        siapa yang terlibat. Saya memastikan semua temuan didukung oleh bukti yang kuat
        dan melaporkannya sesuai prosedur meskipun ada tekanan dari berbagai pihak.
        Saya konsisten menerapkan nilai-nilai integritas dalam setiap tindakan dan keputusan.
        """,
        "kompetensi": "Integritas",
        "level_target": 4
    },
    {
        "nama": "Ahmad Hidayat",
        "jabatan": "Kepala Bagian Tata Usaha",
        "jawaban_test": """
        Dalam menyampaikan informasi penting kepada seluruh unit kerja, saya menggunakan 
        berbagai saluran komunikasi yang efektif seperti email resmi, rapat koordinasi, 
        dan media internal. Saya memastikan pesan disampaikan dengan jelas, tepat waktu, 
        dan mudah dipahami oleh semua pihak. Saya juga selalu meminta feedback untuk 
        memastikan informasi tersampaikan dengan baik.
        """,
        "kompetensi": "Komunikasi",
        "level_target": 3
    },
    {
        "nama": "Dewi Kartika",
        "jabatan": "Kepala Subbagian Protokol",
        "jawaban_test": """
        Dalam melaksanakan tugas keprotokolan, saya selalu memastikan setiap acara 
        berjalan sesuai standar dan tepat waktu. Saya menyusun rencana kerja yang detail,
        mengalokasikan sumber daya secara efisien, dan melakukan monitoring berkala 
        untuk memastikan target tercapai. Saya juga melakukan evaluasi pasca acara 
        untuk perbaikan di masa depan.
        """,
        "kompetensi": "Orientasi pada hasil",
        "level_target": 2
    },
    {
        "nama": "Rudi Hermawan",
        "jabatan": "Pimpinan Tinggi Pratama - Kepala PPSDM",
        "jawaban_test": """
        Ketika organisasi menghadapi perubahan struktur dan kebijakan baru, saya 
        memimpin tim untuk memahami perubahan tersebut dengan menyelenggarakan sosialisasi
        dan pelatihan. Saya mengidentifikasi dampak perubahan, menyusun strategi adaptasi,
        dan memastikan semua anggota tim siap menghadapi transisi. Saya juga membangun
        budaya organisasi yang mendukung perubahan positif.
        """,
        "kompetensi": "Mengelola Perubahan",
        "level_target": 4
    }
]


for i, person in enumerate (test_cases, 1): 
    print(f"Penilaian ke {i} dengan nama {person["nama"]}")
    print(f"Jabatan {person["jabatan"]}")
    print("="*60)

    hasil_penilaian = competency_asessment(
        nama=person["nama"],
        jabatan=person["jabatan"],
        jawaban_test=person["jawaban_test"],
        kompetensi=person["kompetensi"],
        level_target=person["level_target"]
    )

    print("Hasil Penilaian")
    print(hasil_penilaian["hasil"])

    print("\n DOKUMEN SUMBER:")
    print("-" * 30)
    for doc_num, doc in enumerate(hasil_penilaian['sumber'], 1):
        page_label = doc.metadata.get('page_label', 'N/A')
        print(f"  Sumber #{doc_num} (Halaman: {page_label}):")
        print(f"  \"{doc.page_content[:250].strip()}...\"\n") 
        

Penilaian ke 1 dengan nama Budi Santoso
Jabatan Kepala Bagian Pengadaan Barang dan Jasa


ValueError: Missing some input keys: {'skj_terpilih', 'context_permenpan', 'context_skj'}